In [29]:
# ✅ FINAL STAGE 2: LangGraph Definition Using Real Agents (CustomerAgent, InventoryAgent, PricingAgent, SupplyChainAgent)

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langchain_core.runnables import Runnable
from pydantic import BaseModel
from datetime import datetime


In [30]:
# ----------------------------
# ✅ 1. Define State Schema
# ----------------------------
class InventoryState(BaseModel):
    product_id: str
    date: str
    store_id: int
    predicted_sales: float
    stock_levels: int
    reorder_point: int
    price: float
    promotions: int
    reorder_decision: bool
    reorder_quantity: int
    expected_delivery_days: int

In [31]:
# ----------------------------
# ✅ 2. Define Real Agent Functions
# ----------------------------
# Make sure these are copied from previous real agent definitions you implemented
from typing import Dict
import pandas as pd

# Load files globally for reuse
pricing_df = pd.read_csv("processed_pricing.csv")
inventory_df = pd.read_csv("processed_inventory.csv")


In [35]:
# Load the trained forecasting model
with open("sales_forecast_model.pkl", "rb") as f:
    forecasting_model = pickle.load(f)

# Define State Schemas
class CustomerState(TypedDict):
    product_id: str
    date: str
    store_id: int
    sales_quantity: int
    price: float
    promotions: int
    predicted_sales: float

class InventoryState(TypedDict):
    product_id: str
    date: str
    store_id: int
    price: float
    promotions: int
    predicted_sales: float
    stock_levels: float
    reorder_point: float
    reorder_decision: str
    reorder_quantity: float
    expected_delivery_days: int

class PricingState(TypedDict):
    product_id: str
    store_id: int
    price: float
    predicted_sales: float
    reorder_decision: str
    reorder_quantity: float
    new_price: float

class SupplyChainState(TypedDict):
    product_id: str
    store_id: int
    reorder_decision: str
    reorder_quantity: float
    expected_delivery_days: int
    supplier_lead_time_days: float
    warehouse_capacity: float
    final_decision: str


# Define Agents

def customer_agent(data: dict) -> CustomerState:
    features = [[
        data['price'],
        data['promotions'],
        data['sales_quantity'],
        data['sales_7d_avg']
    ]]
    prediction = forecasting_model.predict(features)[0]
    return {
        'product_id': str(data['product_id']),
        'date': data['date'],
        'store_id': data['store_id'],
        'sales_quantity': data['sales_quantity'],
        'price': data['price'],
        'promotions': data['promotions'],
        'predicted_sales': prediction
    }

def prepare_inventory_state(state: CustomerState) -> InventoryState:
    # Get matching inventory row
    row = inventory_df[
        (inventory_df['product_id'] == int(state['product_id'])) &
        (inventory_df['store_id'] == state['store_id'])
    ].iloc[0]

    return {
        **state,
        'stock_levels': row['stock_levels'],
        'reorder_point': row['reorder_point'],
        'reorder_decision': '',  # placeholder
        'reorder_quantity': 0.0,  # placeholder
        'expected_delivery_days': 0  # placeholder
    }

def inventory_agent(state: InventoryState) -> InventoryState:
    reorder_decision = 'Yes' if state['stock_levels'] < state['reorder_point'] else 'No'
    reorder_quantity = max(0, state['reorder_point'] - state['stock_levels'])
    expected_days = 5 if reorder_decision == 'Yes' else 0

    state.update({
        'reorder_decision': reorder_decision,
        'reorder_quantity': reorder_quantity,
        'expected_delivery_days': expected_days
    })
    return state

def pricing_agent(state: InventoryState) -> PricingState:
    if state['reorder_decision'] == 'No':
        new_price = state['price']
    else:
        new_price = round(state['price'] * 1.05, 2)  # 5% markup

    return {
        'product_id': state['product_id'],
        'store_id': state['store_id'],
        'price': state['price'],
        'predicted_sales': state['predicted_sales'],
        'reorder_decision': state['reorder_decision'],
        'reorder_quantity': state['reorder_quantity'],
        'new_price': new_price
    }

def supply_chain_agent(state: InventoryState) -> SupplyChainState:
    row = inventory_df[
        (inventory_df['product_id'] == int(state['product_id'])) &
        (inventory_df['store_id'] == state['store_id'])
    ].iloc[0]

    if state['reorder_decision'] == 'Yes' and row['warehouse_capacity'] > state['reorder_quantity']:
        final_decision = "Proceed with reorder"
    else:
        final_decision = "Hold reorder"

    return {
        'product_id': state['product_id'],
        'store_id': state['store_id'],
        'reorder_decision': state['reorder_decision'],
        'reorder_quantity': state['reorder_quantity'],
        'expected_delivery_days': state['expected_delivery_days'],
        'supplier_lead_time_days': row['supplier_lead_time_(days)'],
        'warehouse_capacity': row['warehouse_capacity'],
        'final_decision': final_decision
    }





In [36]:
# ----------------------------
# ✅ 3. Create LangGraph Workflow
# ----------------------------
# LangGraph Setup
graph = StateGraph(InventoryState)

# Define flow
graph.add_node("CustomerAgent", customer_agent)
graph.add_node("PrepareInventoryState", prepare_inventory_state)
graph.add_node("InventoryAgent", inventory_agent)
graph.add_node("PricingAgent", pricing_agent)
graph.add_node("SupplyChainAgent", supply_chain_agent)

# Define edges
graph.set_entry_point("CustomerAgent")
graph.add_edge("CustomerAgent", "PrepareInventoryState")
graph.add_edge("PrepareInventoryState", "InventoryAgent")
graph.add_edge("InventoryAgent", "PricingAgent")
graph.add_edge("PricingAgent", "SupplyChainAgent")
graph.add_edge("SupplyChainAgent", END)

# Compile the graph
final_graph = graph.compile()

# Sample input to test
sample = {
    'product_id': '1000',
    'date': '2024-09-04',
    'store_id': 22,
    'sales_quantity': 245,
    'price': 60.79,
    'promotions': 0,
    'sales_7d_avg': 245.0
}

result = final_graph.invoke(sample)
print(result)

KeyError: 'sales_quantity'

In [43]:
import pandas as pd
import joblib
from pydantic import BaseModel

# Load model
model = joblib.load("sales_forecast_model.pkl")

# Load full dataset (needed for one-hot encoding)
df = pd.read_csv("processed_demand.csv")

# Extract date features (they were used during training)
df["date"] = pd.to_datetime(df["date"])
df["day_of_week"] = df["date"].dt.dayofweek
df["month"] = df["date"].dt.month
df["week"] = df["date"].dt.isocalendar().week

# One-hot encode the correct features
categorical_cols = ["seasonality_factors", "external_factors", "demand_trend", "customer_segments"]
df_encoded = pd.get_dummies(df, columns=categorical_cols, prefix_sep="__")
df_encoded.columns = [col.replace("__", " ") for col in df_encoded.columns]

# Drop unused columns
df_encoded = df_encoded.drop(columns=["date"])

# Grab first row for testing
test_row = df_encoded.iloc[0]

# Define schema matching model inputs
class CustomerState(BaseModel):
    product_id: int
    store_id: int
    price: float
    promotions: int
    sales_7d_avg: float
    # The rest are optional because they depend on one-hot result
    seasonality_factors_Festival: float = 0
    seasonality_factors_Holiday: float = 0
    seasonality_factors_None: float = 0
    external_factors_Competitor_Pricing: float = 0
    external_factors_Economic_Indicator: float = 0
    external_factors_None: float = 0
    external_factors_Weather: float = 0
    demand_trend_Decreasing: float = 0
    demand_trend_Increasing: float = 0
    demand_trend_Stable: float = 0
    customer_segments_Budget: float = 0
    customer_segments_Premium: float = 0
    customer_segments_Regular: float = 0
    day_of_week: int
    month: int
    week: int

# Define agent logic
def customer_agent(state: CustomerState) -> CustomerState:
    input_data = pd.DataFrame([state.model_dump()])
    predicted_sales = model.predict(input_data)[0]

    updated_state = state.model_copy()
    print("✅ Predicted Sales:", predicted_sales)
    return updated_state

# Create state from real encoded row
features = model.get_booster().feature_names
filtered_row = test_row[features]

# Rename feature names to match Pydantic (XGBoost removes special chars in dummies)
filtered_row.index = [col.replace(" ", "_") for col in filtered_row.index]

# Convert to dictionary for Pydantic model
data_dict = filtered_row.to_dict()

# Convert float32 to float (Pydantic strict typing)
data_dict = {k: float(v) if isinstance(v, float) else v for k, v in data_dict.items()}

# Instantiate and run
try:
    state = CustomerState(**data_dict)
    result = customer_agent(state)
    print("✅ Output:\n", result.model_dump())
except Exception as e:
    print("❌ Error:", e)


KeyError: "['seasonality_factors_Festival', 'seasonality_factors_Holiday', 'seasonality_factors_None', 'external_factors_Competitor Pricing', 'external_factors_Economic Indicator', 'external_factors_None', 'external_factors_Weather', 'demand_trend_Decreasing', 'demand_trend_Increasing', 'demand_trend_Stable', 'customer_segments_Budget', 'customer_segments_Premium', 'customer_segments_Regular'] not in index"